# Analyse des données pour la sous-nutrition dans le monde

- Proportion de personnes en état de sous-nutrition
- Nombre théorique de personnes qui pourraient être nourries
- Disponibilité alimentaire des produits végétaux
- Nombre de personne théoriquement nourrisable
- L’utilisation de la disponibilité intérieure
- Tonne de nourriture reçu en aide alimentaire
- Disponibilité par habitant
- Pays dont l'importation est supérieur à la production
- Proportion du PIB agricole
- Autres indicateurs

In [98]:
import pandas as pd
import numpy as np

In [99]:
# Mise au propre de la table sous_nutrition
sous_nutrition = pd.read_csv('sous_nutrition.csv')
sous_nutrition = sous_nutrition.fillna(0)

for i in range(0,len(sous_nutrition)):
    if (sous_nutrition['Valeur'][i] != '<0.1'):
        sous_nutrition['Valeur'][i] = int(float(sous_nutrition['Valeur'][i]))*1000000 #1.000.000!
    else:
        sous_nutrition['Valeur'][i] = 0
sous_nutrition['Valeur'] = sous_nutrition['Valeur'].astype(int)
sous_nutrition = sous_nutrition.rename(columns={"Valeur":"sous_nutrition"})
#propre!


In [100]:
# Mise au propre de la table population
population = pd.read_csv('population.csv')

population['Valeur'] = population['Valeur']*1000
population['Valeur'] = population['Valeur'].astype(int)
population = population.rename(columns={'Valeur':'Population'})
#propre!

# La proportion de personnes en état de sous-nutrition

In [101]:
# Selection de l'année 2016-2018 sous_nutrition
sous_nutrition_2017 = sous_nutrition[sous_nutrition['Année'] == '2016-2018']
sous_nutrition_2017 = sous_nutrition_2017.drop(columns={'Année'})

# Selection de l'année 2017 population
population_2017 = population[population['Année'] == 2017]
population_2017 = population_2017.drop(columns={'Année'})

fusion_sn = population_2017.merge(sous_nutrition_2017, how='left', on='Zone')

proportion = (fusion_sn['sous_nutrition'] / fusion_sn['Population'])*100
fusion_sn.insert(3,"Sous-nutrition_%", proportion)

fusion_sn.sort_values(by='Sous-nutrition_%', ascending=False).head(10)

,Zone,Population,sous_nutrition,Sous-nutrition_%
181,République populaire démocratique de Corée,25429825,12000000.0,47.188685
87,Haïti,10982366,5000000.0,45.527530
128,Madagascar,25570512,10000000.0,39.107547
186,Rwanda,11980961,4000000.0,33.386303
216,Tchad,15016753,5000000.0,33.296146
145,Mozambique,28649018,9000000.0,31.414689
0,Afghanistan,36296113,10000000.0,27.551160
231,Venezuela (République bolivarienne du),29402484,8000000.0,27.208586
203,Sierra Leone,7488423,2000000.0,26.707893
106,Iraq,37552781,9000000.0,23.966268


# Le nombre théorique de personnes qui pourraient être nourries toutes denrée alimentaire confondu

Le choix des calories est basé sur une moyenne de 2200 kcal/j elle varie selon les avis scientifique et l'activité des personnes concerné. Par exemple un militaire en mission est considéré à 3000 kcal/j quelqu'un ne faisant rien peut se contenter de 1900 kcal/j

https://www.lepoint.fr/monde/la-fao-defend-ses-chiffres-de-la-faim-29-11-2016-2086634_24.php

In [112]:
#mise au propre
dispo_alimentaire = pd.read_csv('dispo_alimentaire.csv')
dispo_alimentaire = dispo_alimentaire.rename(columns={'kg/personne/an':'Kg/pers/an'})
dispo_alimentaire = dispo_alimentaire.rename(columns={'Kcal/personne/jour':'Kcal/pers/j'})
#propre!

dispo_Kcal = dispo_alimentaire.loc[:,['Zone','Kcal/pers/j']]
dispo_Kcal['Kcal/pers/j'] = dispo_alimentaire['Kcal/pers/j'].dropna()

Kcal = dispo_Kcal.groupby(by=['Zone']).sum()
fusion_kcal = population_2017.merge(Kcal, how='inner', on='Zone')

fusion_kcal.insert(3,'nourrissable',((fusion_kcal['Population']*fusion_kcal['Kcal/pers/j'])/2200).astype(int))
fusion_kcal.insert(4,'nourrissable_%',((fusion_kcal['nourrissable']/fusion_kcal['Population'])*100))

fusion_kcal.sort_values(by='Kcal/pers/j', ascending=True).head(20)

,Zone,Population,Kcal/pers/j,nourrissable,nourrissable_%
131,République centrafricaine,4596023,1879.0,3925421,85.409081
170,Zambie,16853599,1924.0,14739238,87.454543
97,Madagascar,25570512,2056.0,23896805,93.454542
0,Afghanistan,36296113,2087.0,34431812,94.863635
70,Haïti,10982366,2089.0,10428255,94.954539
136,République populaire démocratique de Corée,25429825,2093.0,24193010,95.136361
156,Tchad,15016753,2109.0,14395605,95.863633
171,Zimbabwe,14236595,2113.0,13673602,96.045452
120,Ouganda,41166588,2126.0,39781893,96.636362
55,Éthiopie,106399924,2129.0,102966108,96.772727


# La disponibilité alimentaire des produits végétaux et
# Le nombre de personne nourrisable avec des végétaux

In [103]:
dispo_veg = dispo_alimentaire[dispo_alimentaire['Origine'] == "vegetale"]
dispo_veg = dispo_veg.loc[:,['Zone','Kcal/pers/j','Kg/pers/an']]
dispo_veg = dispo_veg.groupby(by=['Zone']).sum()
fusion_veg = population_2017.merge(dispo_veg, how='inner', on='Zone')

fusion_veg.insert(4,'nourrissable',((fusion_veg['Population']*fusion_veg['Kcal/pers/j'])/2200).astype(int))
fusion_veg.insert(5,'nourrissable_%',((fusion_veg['nourrissable']/fusion_veg['Population'])*100))

fusion_veg.sort_values(by='Kcal/pers/j', ascending=True).head(10)

,Zone,Population,Kcal/pers/j,Kg/pers/an,nourrissable,nourrissable_%
6,Antigua-et-Barbuda,95426,1635.0,420.14,70918,74.317272
131,République centrafricaine,4596023,1673.0,398.32,3495066,76.045442
107,Mongolie,3113786,1679.0,340.78,2376384,76.318154
50,Équateur,16785361,1732.0,327.21,13214656,78.727267
21,Bolivie (État plurinational de),11192855,1790.0,390.43,9106913,81.363629
170,Zambie,16853599,1818.0,316.47,13927201,82.636362
140,Sainte-Lucie,180954,1820.0,387.25,149698,82.727102
141,Saint-Kitts-et-Nevis,52045,1821.0,343.60,43079,82.772601
150,Soudan,40813397,1824.0,376.19,33838016,82.909090
13,Bahamas,381755,1828.0,588.37,317203,83.090726


# Denrées animals et végétale mais sans les alcools, graisses saturées et sucres

In [104]:
dispo_filtre = dispo_alimentaire

ban_list = {"Alcool, non Comestible","Bière","Boissons Alcooliques","Edulcorants Autres","Graisses Animales Crue","Sucre Eq Brut","Sucre non centrifugé","Sucre, betterave","Sucre, canne","Vin"}
dispo_filtre = dispo_filtre.loc[-dispo_filtre["Produit"].isin(ban_list)]
#Filtre appliqué

dispo_filtre = dispo_filtre.loc[:,['Zone','Kcal/pers/j','Kg/pers/an']]
dispo_filtre = dispo_filtre.groupby(by=['Zone']).sum()
dispo_filtre = population_2017.merge(dispo_filtre, how='inner', on='Zone')

dispo_filtre.insert(4,'nourrissable',((dispo_filtre['Population']*dispo_filtre['Kcal/pers/j'])/2200).astype(int))
dispo_filtre.insert(5,'nourrissable_%',((dispo_filtre['nourrissable']/dispo_filtre['Population'])*100))

dispo_filtre.sort_values(by='Kcal/pers/j', ascending=True).head(10)

,Zone,Population,Kcal/pers/j,Kg/pers/an,nourrissable,nourrissable_%
171,Zimbabwe,14236595,1753.0,279.94,11343977,79.681813
170,Zambie,16853599,1806.0,328.42,13835272,82.090905
131,République centrafricaine,4596023,1812.0,445.37,3785451,82.363622
70,Haïti,10982366,1837.0,367.47,9170275,83.499994
111,Namibie,2402633,1838.0,497.51,2007290,83.545427
21,Bolivie (État plurinational de),11192855,1884.0,450.79,9585154,85.636364
66,Guatemala,16914970,1906.0,383.75,14654514,86.636358
169,Yémen,27834819,1931.0,346.05,24431379,87.772725
65,Grenade,110874,1942.0,522.66,97871,88.272273
141,Saint-Kitts-et-Nevis,52045,1946.0,465.04,46036,88.454222


# L’utilisation de la disponibilité intérieure, et les proportions pour chaque utilisation

Intérieur = Production + Importation - Exportation + variation de stock

In [105]:
dispo_animaux = dispo_alimentaire.groupby(by=['Zone']).sum()
dispo_animaux = dispo_animaux.loc[:,['intérieure','Nourriture','Aliments pour animaux','Autres Utilisations','Pertes','Semences','Traitement']]

dispo_animaux['Traitement'] = (dispo_animaux['Traitement'] / dispo_animaux['intérieure'])*100
dispo_animaux['Semences'] = (dispo_animaux['Semences'] / dispo_animaux['intérieure'])*100
dispo_animaux['Pertes'] = (dispo_animaux['Pertes'] / dispo_animaux['intérieure'])*100
dispo_animaux['Autres Utilisations'] = (dispo_animaux['Autres Utilisations'] / dispo_animaux['intérieure'])*100
dispo_animaux['Aliments pour animaux'] = (dispo_animaux['Aliments pour animaux'] / dispo_animaux['intérieure'])*100
dispo_animaux['Nourriture'] = (dispo_animaux['Nourriture'] / dispo_animaux['intérieure'])*100

dispo_animaux = dispo_animaux.rename(columns={'Traitement':'Traitement_%'})
dispo_animaux = dispo_animaux.rename(columns={'Nourriture':'Nourriture_%'})
dispo_animaux = dispo_animaux.rename(columns={'Aliments pour animaux':'Animaux_%'})
dispo_animaux = dispo_animaux.rename(columns={'Autres Utilisations':'Autre_%'})
dispo_animaux = dispo_animaux.rename(columns={'Pertes':'Pertes_%'})
dispo_animaux = dispo_animaux.rename(columns={'Semences':'Semences_%'})

dispo_animaux = dispo_animaux.merge(fusion_sn.drop(columns=['Population','sous_nutrition']), how='left', on='Zone')
dispo_animaux.sort_values(by='Nourriture_%', ascending=False).head(10)

,Zone,intérieure,Nourriture_%,Animaux_%,Autre_%,Pertes_%,Semences_%,Traitement_%,Sous-nutrition_%
34,Chine - RAS de Macao,383.0,96.083551,0.000000,2.610966,0.261097,0.000000,0.783290,0.000000
154,Timor-Leste,447.0,93.736018,2.013423,0.447427,3.131991,0.447427,0.671141,0.000000
165,Yémen,9782.0,93.661828,2.146800,0.736046,2.749949,0.684932,0.398691,0.000000
84,Lesotho,966.0,93.064182,0.103520,0.000000,5.279503,0.517598,1.035197,0.000000
127,Rwanda,10054.0,92.231947,0.000000,0.188980,4.326636,1.641138,1.661030,33.386303
99,Mauritanie,2142.0,90.896359,4.154995,0.793651,3.501401,0.560224,0.000000,0.000000
33,Chine - RAS de Hong-Kong,6424.0,89.943960,1.556663,4.856787,3.362391,0.031133,0.171233,0.000000
21,Botswana,1253.0,88.826816,1.915403,2.713488,4.070231,0.239425,1.835595,0.000000
22,Brunéi Darussalam,294.0,88.775510,3.061224,5.102041,1.360544,0.340136,0.000000,0.000000
18,Bermudes,59.0,86.440678,1.694915,37.288136,0.000000,0.000000,0.000000,0.000000


# Tonne de nourriture reçu en aide alimentaire

In [106]:
aide = pd.read_csv('aide_alimentaire.csv')
aide = aide.rename(columns={'Pays bénéficiaire':'Zone'})
aide = aide.drop(columns={'Année'})
aide = aide.groupby(by=['Zone']).sum()
aide = aide.rename(columns={'Valeur':'Nourriture_Tonne'})
aide = aide.merge(fusion_sn.drop(columns=['Population','sous_nutrition']), how='left', on='Zone')
aide.sort_values(by='Nourriture_Tonne', ascending=False).head(50)

,Zone,Nourriture_Tonne,Sous-nutrition_%
50,République arabe syrienne,1858943,0.000000
75,Éthiopie,1381294,19.736856
70,Yémen,1206484,0.000000
61,Soudan du Sud,695248,0.000000
60,Soudan,669784,12.250879
30,Kenya,552836,21.903126
3,Bangladesh,348188,13.150856
59,Somalie,292678,0.000000
53,République démocratique du Congo,288502,0.000000
43,Niger,276344,0.000000


# Disponibilité par habitant

In [107]:
dispo_habitant = dispo_alimentaire.groupby(by=['Zone']).sum()
dispo_habitant = population_2017.merge(dispo_habitant, how='left', on='Zone')
dispo_habitant = dispo_habitant.rename(columns={'Valeur':'Population'})
dispo_habitant = dispo_habitant.loc[:,['Zone','Population','Nourriture']]
dispo_habitant.insert(3,'dispo/hab_Kg',dispo_habitant['Nourriture']/dispo_habitant['Population']*1000000)

dispo_habitant.sort_values(by='dispo/hab_Kg', ascending=True).head(10)

,Zone,Population,Nourriture,dispo/hab_Kg
216,Tchad,15016753,4097.0,272.828620
0,Afghanistan,36296113,10735.0,295.761698
234,Zambie,16853599,5088.0,301.893975
66,Éthiopie,106399924,33361.0,313.543457
72,Gambie,2213889,701.0,316.637374
233,Yémen,27834819,9162.0,329.156083
84,Guinée-Bissau,1828145,609.0,333.124561
235,Zimbabwe,14236595,4762.0,334.490094
219,Timor-Leste,1243258,419.0,337.017739
200,Sénégal,15419355,5329.0,345.604599


# Pays dont l'importation est supérieur à la production

La chine produit sa nourriture à l'étranger en achetant des terres agricoles

https://www.lepoint.fr/monde/la-chine-a-la-conquete-des-terres-agricoles-23-02-2018-2197309_24.php

In [109]:
ratio_import_prod = dispo_alimentaire.groupby(by=['Zone']).sum().loc[:,['intérieure','Production','Importations','Exportations']]
ratio_import_prod = ratio_import_prod[ratio_import_prod['Importations'] > ratio_import_prod['Production']]
ratio_import_prod = ratio_import_prod.merge(fusion_sn.drop(columns=['Population','sous_nutrition']), how='left', on='Zone')
ratio_import_prod.sort_values(by='Sous-nutrition_%', ascending=False).head(60)

,Zone,intérieure,Production,Importations,Exportations,Sous-nutrition_%
1,Arabie saoudite,31274.0,8878.0,27726.0,4707.0,3.021041
0,Antigua-et-Barbuda,79.0,20.0,63.0,0.0,0.000000
28,Yémen,9782.0,4125.0,6786.0,719.0,0.000000
27,Trinité-et-Tobago,1072.0,275.0,898.0,95.0,0.000000
26,Slovénie,2664.0,1854.0,1915.0,1192.0,0.000000
25,Sainte-Lucie,136.0,68.0,84.0,19.0,0.000000
24,Polynésie française,335.0,159.0,176.0,10.0,0.000000
23,Pays-Bas,42461.0,41636.0,52910.0,52179.0,0.000000
22,Oman,3518.0,1169.0,3464.0,1217.0,0.000000
21,Nouvelle-Calédonie,272.0,78.0,201.0,5.0,0.000000


# Proportion du PIB agricole

In [95]:
PIB_agricole = pd.read_csv('PIB_agricole.csv')
PIB_agricole = fusion_sn.drop(columns=['Population','sous_nutrition']).merge(PIB_agricole, how='left', on='Zone')
PIB_agricole = PIB_agricole.rename(columns={'2017':'PIB_agricole_%'})

PIB = pd.read_csv('PIB.csv')
PIB = PIB.fillna(1)
PIB['PIB'] = PIB['PIB'].astype(float)/1000000
PIB = PIB.rename(columns={'PIB':'PIB en million_$'})
PIB_agricole = PIB_agricole.merge(PIB, how='left', on='Zone')

PIB_agricole.sort_values(by='PIB_agricole_%', ascending=False).head(10)

,Zone,Sous-nutrition_%,PIB_agricole_%,PIB en million_$
203,Sierra Leone,26.707893,60.611090,3719.369107
84,Guinée-Bissau,0.000000,49.157808,1350.177128
216,Tchad,33.296146,48.613086,10000.395242
174,République arabe syrienne,0.000000,40.789491,16340.665432
132,Mali,5.401776,37.432440,15365.713059
122,Libéria,21.266524,35.942616,3390.703400
151,Niger,0.000000,35.816877,11185.102400
66,Éthiopie,19.736856,33.779281,81770.791971
175,République centrafricaine,0.000000,32.790171,2072.349974
47,Comores,0.000000,31.898887,1077.439663


# Autres indicateurs

In [96]:
Depense_pub = pd.read_csv('Dépense_publique.csv')
Depense_pub = population_2017.merge(Depense_pub, how='left', on='Zone')

dispo_alimentaire = dispo_alimentaire.groupby(by=['Zone']).sum().loc[:,['intérieure']]
Depense_pub = Depense_pub.merge(dispo_alimentaire, how='left', on='Zone')

Depense_pub = Depense_pub.merge(fusion_sn.drop(columns=['Population','sous_nutrition']), how='left', on='Zone')

Depense_pub['intérieure'] = (Depense_pub['intérieure']*1000000)/Depense_pub['Population']
Depense_pub = Depense_pub.rename(columns={'intérieure':'Kg/hab/an'})

Depense_pub = Depense_pub.merge(PIB_agricole.loc[:,['Zone','PIB_agricole_%']],how='left',on='Zone')
Depense_pub = Depense_pub.merge(PIB, how='left', on='Zone')

Depense_pub.insert(3,'dépense/hab_$',(Depense_pub['dépense_M_$']*1000000)/Depense_pub['Population'])

Depense_pub.sort_values(by='Sous-nutrition_%', ascending=False).head(60)

,Zone,Population,dépense_M_$,dépense/hab_$,Kg/hab/an,Sous-nutrition_%,PIB_agricole_%,PIB en million_$
181,République populaire démocratique de Corée,25429825,NaN,NaN,584.982398,47.188685,NaN,NaN
87,Haïti,10982366,NaN,NaN,562.264998,45.527530,18.898831,1.503556e+04
128,Madagascar,25570512,79.204407,3.097490,595.060435,39.107547,24.550138,1.317631e+04
186,Rwanda,11980961,63.929200,5.335899,839.164738,33.386303,26.345456,9.252834e+03
216,Tchad,15016753,NaN,NaN,353.205517,33.296146,48.613086,1.000040e+04
145,Mozambique,28649018,79.339698,2.769369,768.717448,31.414689,25.043681,1.321908e+04
0,Afghanistan,36296113,288.423438,7.946400,372.353921,27.551160,26.420199,1.875347e+04
231,Venezuela (République bolivarienne du),29402484,NaN,NaN,1098.104500,27.208586,NaN,NaN
203,Sierra Leone,7488423,1.070381,0.142938,877.354284,26.707893,60.611090,3.719369e+03
106,Iraq,37552781,NaN,NaN,578.465813,23.966268,2.976728,1.872177e+05


# Point d'amélioration et piste de recherche

- **Enlever les alcools de la dispo alimentaire**: 
L'alcool est mauvais pour la santé au même titre que le tabac et il fait augmenter la quantité de calorie en flèche; idem pour les matière grasse saturé et le sucre.

 
- **Mesurer la qualité moyenne des aliments**:
Un certain nombre d'étude nous démontre que le taux de vitamines dans nos fruits et légumes sont en chute depuis 
les années 50 ce qui correspond aux débuts de l'agriculture intensive chimique.

 
- **Prendre en compte la vulnérabilité du climat**:
Non seulement parce qu'il est difficile de faire pousser un potager dans le desert mais aussi parce que toute les terres ne sont pas toutes cultivable. Une terre sableuse demande d'être arrosé très souvent et nécessite un apport en engrais plus concéquant qu'une terre vivante pleine d'humus et de verre de terre.


- **Rétablir les données sur le niveau de mécanisation de l'agriculture**. 
Elles sont actuellement abandonné et archivé depuis 2009.
La mécanisation de l'agriculture, est, ce qui permet à un pays de libérer du temps de travail, pour passer d'une scociété agricole à une scociété industrielle comme nous l'avons fait en occident depuis le 19ème sciècle.


Cela permet de **réduire le coût de production de la nourriture** et donc de pratiquer des prix moins chère rendant les denrées alimentaire **accessible aux plus pauvre**. Cela permet de rendre cette partie de la population employable et productive.

Ce faisant un cercle vertueux peut commencer pour les pays en voie de développement.
Le secteur agricole réclament **moins de main d'oeuvre** de nouveau métiers peuvent enfin émerger permettant au pays de **diversifier ses activités économiques** les rendants moins fragile aux crises financières.

L'éducation et l'acquisition de diplôme deviens alors une nécessité pour travailler, obligeant la population à s'instruire.
**L'instructions des femmes** permet de réduire les **démographies explosives** et ainsi la quantité de denrée alimentaire nécessaire aux fonctionnements du pays. Bien évidement ce n'est pas le seul bénéfice cela influe aussi sur la croissance économique et tout un tas de bonne chose.
https://www.sciencespo.fr/research/cogito/home/education-des-filles-technologies-et-transition-demographique/

**Conclusion**: 
- Les pays en difficultés pour nourrire leur population, sont des pays à l'activité économique peu diversifié.
- On peut toujours chercher un indicateur principale il y aura toujours des cas particuliers pour nous contredire.
- Les aides alimentaires sont des morceaux de scotch qui ne résouds pas le problème dans son ensemble.
- Le développement économique doit se faire de l'intérieur et non avec des machines importé, **le besoin d'ingénieur de main d'oeuvre compétente sera la clée de la souveraineté alimentaire des pays**.
Nous le voyons aujourd'hui avec la guerre en Ukraine et la crise de la COVID-19. Même les pays développé ont été victime de leur politique de délocalisation des industries et donc de la mondialisation des activités économique.